### MacroMicro 사이트에 나온 데이터들을 크롤링 하는 코드

In [1]:
# 필요한 라이브러리들을 로드하자
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager

#### MacroMicro 사이트에서 데이터들을 크롤링 하려면 웹드라이버에서 구글 계정으로 로그인 하는게 필요함

In [2]:
# 크롬 웹드라이버를 실행해서 로그인하는 과정을 함수로 나타낸거

def init_driver():
    driver = uc.Chrome()
    driver.get('https://velog.io')
    return driver
# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(driver):
    driver.find_element(By.XPATH,'//button[text()="로그인"]').click()
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located(
            (By.XPATH, '//button[text()="새 글 작성"]')
        )
    )

In [3]:
# main에서 실행하지 않으면 오류가 남
# https://github.com/ultrafunkamsterdam/undetected-chromedriver/issues/486#issuecomment-1032009193 참조
# 크롬 웹드라이버를 실행하고 구글 계정으로 로그인하자
if  __name__  ==  "__main__" :
    driver = init_driver()
    do_login(driver)

In [4]:
# G10 ESI 데이터가 있는 곳을 웹드라이버로 실행시켜보자
url = 'https://en.macromicro.me/charts/45866/global-citi-surprise-index'  # 원하는 웹사이트 주소로 변경하세요
driver.get(url)

In [5]:
# 로그인 창이 뜰거임. 로그인 하기위한 첫 번째 과정
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login?next=%2Fcharts%2F45866%2Fglobal-citi-surprise-index"]'))
)

# 찾은 요소를 클릭합니다.
element.click()

In [6]:
# 로그인 하기위한 두 번째 과정
element2 = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login/google"]'))
)

# 찾은 요소를 클릭합니다.
element2.click()

In [7]:
# 우리가 관심 있는거는 G10 ESI뿐. 그래서 다른 것들은 나타나지 않도록 해두자
_list = [0,2,3,4,5,6,7]
for i in _list:
    element = driver.find_element(By.CSS_SELECTOR, f'.highcharts-legend-item.highcharts-line-series.highcharts-color-undefined.highcharts-series-{i}')
    element.click()

In [8]:
# 마우스를 움직이고 클릭하기 위한 액션을 정의해줌
actions = ActionChains(driver)

이상하게 커서 위치 초기 설정하는거랑 그 다음 코드는 두번 정도 실행해야 하더라....

In [14]:
# 내가 수동으로 찾은 초기 커서 위치로서 적합한 커서 위치
element3 = driver.find_element(By.CLASS_NAME, 'highcharts-axis-title')

# 찾은 요소를 클릭합니다.
actions.click(element3).perform()   # 우클릭 하고 싶으면 actions.context_click(element3).perform()

In [15]:
# 첫 데이터가 나타나는 곳으로 커서를 이동후 클릭(내가 수동으로 찾음)
actions.move_by_offset(60, 0).click().perform()

#### 참고로 커서 위치는 일일히 찍어먹어 보고 직접 알아내야 함

#### 여기 코드는 웹드라이버 창을 최대화 한 후 화면을 120%까지 확장했을때 기준으로 설정한거. 창의 크기가 달라지면 커서 위치가 조금씩 달라지더라

#### 웹드라이버 창의 크기에 따라, 그리고 각각의 컴퓨터의 크기에 따라 커서를 이동해야하는 정도가 달라지더라... 그래도 초기 커서 위치는 나름 고정 시켜놓긴 함

In [16]:
# 커서를 하나씩 이동 시키면서 커서를 갖다 댈때만 뜨는 데이터 값들을 크롤링 해보자

data_list = []

for i in range(940):

    # 커서를 한 칸씩 옮겨보자
    actions.move_by_offset(1, 0).click().perform()

    # 페이지의 HTML을 가져옴
    html = driver.page_source
    # BeautifulSoup 객체를 생성
    soup = bs(html, 'html.parser')
    # 클래스가 "chart-wrapper"인 요소를 찾아서 해당하는 것을 data에 저장
    wrapper = soup.find('g', attrs={'class': 'highcharts-label highcharts-tooltip highcharts-color-undefined'})
    data = wrapper.prettify()

    # 데이터에서 요일, 월, 일, 연도 추출
    date_start_index = data.find("<tspan style=\"font-size: 10px;\">") + len("<tspan style=\"font-size: 10px;\">")
    date_end_index = data.find("</tspan>")
    date = data[date_start_index:date_end_index].strip()
    # 날짜 잘 나오나 함 찍어보자
    print(date)

    # 데이터에서 G10 ESI 값을 추출
    number_start_index = data.find("<tspan style=\"font-weight: bold;\">") + len("<tspan style=\"font-weight: bold;\">")
    number_end_index = data.find("</tspan>", number_start_index)
    number = data[number_start_index:number_end_index].strip()
    # ESI값 잘 나오나 함 찍어보자
    print(number)

    # 추출한 정보를 리스트에 형태로 하나씩 추가
    data_list.append({'date': date, 'number': number})

# 커서를 하나씩 이동 시켜서 하나씩 크롤링 하는 작업을 끝냈으면 해당 데이터를 데이터 프레임으로 저장
df = pd.DataFrame(data_list)

Wednesday, Jan  1, 2003
-12.30
Wednesday, Jan  1, 2003
-12.30
Wednesday, Jan  1, 2003
-12.30
Monday, Jan  6, 2003
-6.30
Wednesday, Jan 15, 2003
1.50
Thursday, Jan 23, 2003
7.40
Friday, Jan 31, 2003
7.20
Friday, Feb  7, 2003
6.90
Monday, Feb 17, 2003
18.80
Monday, Feb 24, 2003
-0.20
Tuesday, Mar  4, 2003
5.30
Wednesday, Mar 12, 2003
1.50
Thursday, Mar 20, 2003
-5.80
Friday, Mar 28, 2003
-7.40
Friday, Apr  4, 2003
-21.60
Monday, Apr 14, 2003
-10.10
Monday, Apr 21, 2003
-18.00
Tuesday, Apr 29, 2003
-17.80
Wednesday, May  7, 2003
-27.30
Thursday, May 15, 2003
-43.40
Friday, May 23, 2003
-37.80
Friday, May 30, 2003
-26.80
Monday, Jun  9, 2003
-19.60
Monday, Jun 16, 2003
-16.00
Tuesday, Jun 24, 2003
-7.90
Wednesday, Jul  2, 2003
-1.20
Thursday, Jul 10, 2003
-4.20
Friday, Jul 18, 2003
11.90
Friday, Jul 25, 2003
11.00
Monday, Aug  4, 2003
14.80
Monday, Aug 11, 2003
25.80
Tuesday, Aug 19, 2003
38.40
Wednesday, Aug 27, 2003
42.50
Thursday, Sep  4, 2003
46.60
Friday, Sep 12, 2003
49.50
Friday, Se

KeyboardInterrupt: 

In [23]:
# 생성한 데이터 프레임에서 중복되는 값은 삭제
df_droped = df.drop_duplicates(subset='date')

# 그리고 중복되는 데이터를 제거한 완성된 데이터를 csv의 파일형태로 저장해둔다
df_droped.to_csv('G10 ESI.csv', index=False)

In [12]:
# viewport_width = driver.execute_script("return document.documentElement.clientWidth")
# viewport_height = driver.execute_script("return document.documentElement.clientHeight")


# # 9시 방향은 왼쪽 중앙이므로, x 좌표는 0이고 y 좌표는 뷰포트 높이의 절반입니다.
# x = 0
# y = viewport_height / 2